# DS 653 -- Homework 3

**Due:** Saturday, February 15 at 10pm on [Gradescope](https://www.gradescope.com/courses/959425).

_You must follow the Academic Code of Conduct and Collaboration Policy stated in the course syllabus at all times while working on this assignment._

This assignment contains 5 questions worth a total of 24 points. You will earn:
- 1 course learning credit for earning at least 10 points, or
- 2 course learning credits for earning at least 20 points on this homework.

_If you write code that attempts to fool the tests rather than solving the actual question, then you will receive a 0 on the assignment and it will be considered a violation of the Academic Code of Conduct._

This homework is configured to run either on Google Colab or locally on your computer. If you run it locally, please make sure that this Jupyter notebook is the only file in its directory.

To begin, please execute the code block below. It will download this week's auto-grading tests, install and configure otter-grader, and import the crypto and bytestring libraries that we use in this course.

In [17]:
url = "https://crypto-ds.github.io/hw03.zip"

!pip install -q otter-grader && pip install pycryptodomex

# Download zip file containing tests
import os
import urllib.request

zip_file_name = url.split("/")[-1]  # Extract filename from URL
zip_path = os.path.join(os.curdir,zip_file_name)

if not os.path.exists(zip_path):
    print(f"Downloading {url} to {zip_path}...")
    urllib.request.urlretrieve(url, zip_path)
    print("Download complete.")
else:
    print(f"File {zip_path} already exists. Skipping download.")

# Extract test files from zip
import zipfile

extract_dir = os.path.join(os.curdir, "tests") # Where to extract the files
if not os.path.exists(extract_dir):            # Ensure extraction directory exists
    os.makedirs(extract_dir)

print(f"Extracting {zip_path} to {extract_dir}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Unzipped {zip_path} to {extract_dir}")

# Initialize otter-grader
import otter
grader = otter.Notebook()

# Import crypto libraries
import Cryptodome
from binascii import hexlify, unhexlify
from hashlib import sha256

/bin/bash: line 1: /home/annaandmandy/ds653/DS653_crypto/.venv/bin/pip: cannot execute: required file not found


File ./hw03.zip already exists. Skipping download.
Extracting ./hw03.zip to ./tests...
Unzipped ./hw03.zip to ./tests


### Assignment overview

The goal of this week's homework is to learn about the structure of a blockchain. To do so, we introduce a toy blockchain that contains many of the same concepts that are used within Bitcoin, but with some components removed (like the mining rewards, transaction fees, scripting language, and adjustments to the difficulty level).

#### Python classes

Our toy blockchain is written as Python code, which we provide below. We separate the blockchain into four classes:
- The `TransactionPayload` class holds the relevant information for a single payment: the sender and receiver (identified by their public keys) and the amount of money being transferred.
- The `Transaction` class contains the payload together with a digital signature made by the sender.
- The `Block` class contains the block "height" (which is just a 0-indexed counter), a set of new transactions, and a pointer to the previous block in the chain. Additionally, the block contains a `nonce` that can be chosen arbitrarily by the block's miner in order to satisfy the difficulty rule.

Finally, the `blockchain` object is a list of blocks.

Throughout this assignment, the difficulty rule is that:
> _The hash of a valid block must begin with 8 bits (i.e., one byte, or two hex characters) of all 0s_.

To instantiate these Python classes, execute the code blocks below.

In [18]:
## Execute, but DO NOT MODIFY this code block. ##
## It contains a Python class corresponding to a toy blockchain.
## Please read the code to understand how the toy blockchain works.

import json
from Cryptodome.Hash import SHA256
from Cryptodome.PublicKey import ECC
from Cryptodome.Signature import DSS
from binascii import hexlify, unhexlify

class Transaction():
    def __init__(self, transaction_payload, signature):
        self.transaction_payload = transaction_payload # type: TransactionPayload
        self.signature = signature # type: hex-encoded string

    def json_encode(self):
      transaction_json = {"transaction": json.loads(self.transaction_payload.json_encode()), "signature":self.signature}
      return json.dumps(transaction_json)

class TransactionPayload():
    def __init__(self, sender_public_key, receiver_public_key, amount):
        self.sender_public_key = sender_public_key     # these are ECDSA public keys using the Cryptodome library
        self.receiver_public_key = receiver_public_key
        self.amount = amount # type: int

    def json_encode(self):
        return json.dumps(self.__dict__, sort_keys=True)

    def encode(self):
        # encodes the transaction as bytes
        return self.json_encode().encode('ascii')

    def hash(self):
        # returns the hash of the transaction as bytes
        return SHA256.new(self.encode())

class Block():
    def __init__(self, height, transactions, previous_hash, nonce=0):
        self.height = height               # type: int
        self.transactions = transactions   # type: list of transactions
        self.previous_hash = previous_hash # type: hex-encoded string
        self.nonce = nonce                 # type: int

    def json_encode(self):
        json_encoding = {'height': self.height,
                         'transactions': [json.loads(transaction.json_encode()) for transaction in self.transactions],
                         'previous_hash': self.previous_hash, # type: hex-encoded string
                         'nonce': self.nonce
        }
        return json.dumps(json_encoding)

    def encode(self):
        return self.json_encode().encode('ascii')

    def hash(self):
        return SHA256.new(self.encode()).hexdigest()

#### Participants in the toy blockchain

There are five participants who use this toy blockchain. We label these participants as players A, B, C, D, and E. We will only look at a few blocks of this blockchain, which operate as follows:

- Before the start of the blockchain, we assume that players A and C start with 1 coin. The other players have no money at the start.
- The first block contains two transactions: A pays 1 coin to B, and C pays 1 coin to D.
- The second block contains a single transaction: D pays 1 coin to A

After these two blocks, players A and B should have 1 coin each and everyone else should have nothing.

Execute the Python code below to instantiate this blockchain.

In [19]:
## Execute, but DO NOT MODIFY this code block. ##
## This code creates a concrete blockchain containing the transactions and blocks described above.
## It also contains all 5 players' public keys, and the corresponding secret key *only* for Player A.

# Player A's secret key
secret_key = ECC.generate(curve='P-256', randfunc=(lambda x: b'0' * x))
# Public keys for all five players
# Player A's public key is located at index 0, player B's public key is at index 1, and so on
public_keys = ['044d4385fe08e0eb94c524bdd3682c9c5ae358f52402df02418d0ba6cf6889289a27bd56a666df7dc595c98da1f22958009ae17c52fd290185d2e2bf11140a0a5e', '04be4a2555cc8ed29989eeddde3d8e4d41458d02dce047aac2e6af2d58688c5beae7163eb157d1fd3c3d282abab7172b0f0d63274ebe721d31c5d72a83741df170', '04a3fc575afc4ad2ca2cbeed50b52fedf049d317b790aa33e7f8182aec028453082d4c4ffd4c99c7e6d11c134e81020c57ca0fbf0bf3406bf4457dc72d8c994a65', '04abc5bd2786ea5144122ccf43b89a557e1d36e2773c2b8986d9819a22c2e6540b3d5da692504ccc29fbf774435312afdda2a360d5160329cb326f6d47db29f50b', '046a921bed68e07e66f38a5137b3784372cfd4507e6451e1dfe3760f08649750e7891cd30339b730c0280738c60233d3c99eb78454340ff363cafbf1e6f0cfce5e']

blockchain = []

# We assume that A and C magically begin the blockchain with 1 coin.
# This is because our toy blockchain doesn't have mining rewards,
# so we make this assumption to start the chain with some money.

from typing import Union
players_balance = [1,0,1,0,0] # initially, Players A and C have 1 coin each

# Here are the transactions in each block
def gen_block_0():
  # This block sends 1 coin from public key A to public key B
  t1 = TransactionPayload(public_keys[0], public_keys[1], 1)
  # In this toy blockchain, all signatures are stored in hex format and have a type of string
  t1_signature = "edd6bb60e8486f9747f911a6d7340c275d5a417c3823952715a18a64d3577325413cf544e8f48e6c30a312649200c57b72d7eaa4721abf376dbd0dc799db6c3a"
  # A transaction includes the payload and its corresponding signature
  t1_signed = Transaction(t1, t1_signature)

  # C -> D
  t2 = TransactionPayload(public_keys[2], public_keys[3], 1)
  t2_signature = "33bcaaa920207125ec72ea7f9e5fbddb993479dbe7122c6dfcf0a96f2e646dd90ff73c90c784f473480e32fd2fb68735af86458bc0434e5958f025cc9e0ddd59"
  t2_signed = Transaction(t2, t2_signature)

  # block number 0
  previous_hash = "0000000000000000000000000000000000000000000000000000000000000000"
  b0 = Block(0, [t1_signed, t2_signed], previous_hash, 7)
  return b0

def gen_block_1():
  # D -> A
  t3 = TransactionPayload(public_keys[3], public_keys[0], 1)
  t3_signature = "886de552f43010b2a848f660b066102e680a4c6b763032c98c53e6f86c5d5638a9dee5e41f311e4956cd416a51850afa28bc4b9575cbf9cbf1940fae84b1d166"
  t3_signed = Transaction(t3, t3_signature)

  # block number 1
  previous_hash = "00fd71161f8b5244d0641d83ccaedc2f0b7bed9d3f0c625fc817218660c4369a"
  b1 = Block(1, [t3_signed], previous_hash, 143)
  return b1

def pretty_print(blockchain):
  [print(block.json_encode()) for block in blockchain]

blockchain = [gen_block_0(), gen_block_1()]
pretty_print(blockchain)


{"height": 0, "transactions": [{"transaction": {"amount": 1, "receiver_public_key": "04be4a2555cc8ed29989eeddde3d8e4d41458d02dce047aac2e6af2d58688c5beae7163eb157d1fd3c3d282abab7172b0f0d63274ebe721d31c5d72a83741df170", "sender_public_key": "044d4385fe08e0eb94c524bdd3682c9c5ae358f52402df02418d0ba6cf6889289a27bd56a666df7dc595c98da1f22958009ae17c52fd290185d2e2bf11140a0a5e"}, "signature": "edd6bb60e8486f9747f911a6d7340c275d5a417c3823952715a18a64d3577325413cf544e8f48e6c30a312649200c57b72d7eaa4721abf376dbd0dc799db6c3a"}, {"transaction": {"amount": 1, "receiver_public_key": "04abc5bd2786ea5144122ccf43b89a557e1d36e2773c2b8986d9819a22c2e6540b3d5da692504ccc29fbf774435312afdda2a360d5160329cb326f6d47db29f50b", "sender_public_key": "04a3fc575afc4ad2ca2cbeed50b52fedf049d317b790aa33e7f8182aec028453082d4c4ffd4c99c7e6d11c134e81020c57ca0fbf0bf3406bf4457dc72d8c994a65"}, "signature": "33bcaaa920207125ec72ea7f9e5fbddb993479dbe7122c6dfcf0a96f2e646dd90ff73c90c784f473480e32fd2fb68735af86458bc0434e5958f025cc9e0

This `blockchain` turns out to be valid because:
1. The signature in every transaction is correct,
2. Every block has a `nonce` that is chosen to satisfy the difficulty rule,
3. Every block points to the previous block (and by convention, the initial block -- called the "genesis" block -- has a hash of all-0s), and
4. Everyone has enough money in their account at the moment they post a transaction.

If we break any of the four rules, then the blockchain becomes invalid. Here are some examples of invalid blockchains.

In [20]:
# An invalid chain because a digital signature is incorrect
# (specifically, we're altering the signature of the first A -> B transaction)
chain_invalid_sig = blockchain
chain_invalid_sig[0].transactions[0].signature = "00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000"

# An invalid chain because someone is spending money they don't have
# (specifically, we're moving the D -> A transaction into the first block)
chain_invalid_balance = blockchain
chain_invalid_sig[0].transactions[0].transaction_payload = TransactionPayload(public_keys[3], public_keys[0], 1)
chain_invalid_sig[0].transactions[0].signature = "886de552f43010b2a848f660b066102e680a4c6b763032c98c53e6f86c5d5638a9dee5e41f311e4956cd416a51850afa28bc4b9575cbf9cbf1940fae84b1d166"

# An invalid chain with the wrong hash of the previous block
chain_invalid_hash = blockchain
chain_invalid_hash[1].previous_hash = '2222222222222222222222222222222222222222222222222222222222222222'

# An invalid chain that doesn't satisfy the difficulty rule
chain_invalid_difficulty = blockchain
chain_invalid_difficulty[1].nonce = 142

### Programming Questions

For the first 3 questions, your objective is to determine whether any given blockchain is valid according to the 4 rules stated above. If so, you must find the balance held by all five players at the end of the blockchain.

**Question 1: Transaction validity (6 points).** Write the method `isValidTransaction` to verify the correctness of a `transaction` that is provided as input. Your function should return `True` if the transaction is valid and `False` otherwise.

A few reminders to help you out:

- Of the four properties listed above that determine whether a blockchain is valid, only the first property can be checked for an individual transaction. (The other three properties depend on the other contents in the block, or in other blocks of the blockchain, and we will address them in the next two questions.)

- In this toy blockchain, all signatures were created using ECDSA with a NIST-approved curve called `P-256`, and the message that is being signed is the hash of the transaction payload.

In [27]:
def isValidTransaction(transaction: Transaction) -> bool:
    # process raw hex data
    sender_public_key_hex = transaction.transaction_payload.sender_public_key
    point_x = int(sender_public_key_hex[2:66], 16)
    point_y = int(sender_public_key_hex[66:], 16)

    key = ECC.construct(curve = 'P-256', point_x=point_x, point_y=point_y)

    verifier = DSS.new(key, 'fips-186-3')

    h = transaction.transaction_payload.hash()
    signature_bytes = unhexlify(transaction.signature)

    if len(signature_bytes) != 64:
        return False
    
    try:
        verifier.verify(h, signature_bytes)
        return True
    except ValueError:
        return False


In this homework assignment, some of the test cases are *hidden*, which means they will only execute on Gradescope and not on your local/colab setup.

We have provided some public tests as a guide for you as you are writing and debugging your code. But to receive credit, your code must also pass the hidden tests on Gradescope!

In [28]:
grader.check("q1")

q1 results: All test cases passed!

**Question 2: Block validity (6 points).** Next, you must write the method `verifyBlock` to check that the provided `block` has been validly generated. Your function should return `True` if the block is valid and `False` otherwise.

That is: verify that each block satisfies the following two rules.

1. The signature in every transaction is correct, and
2. Every block has a `nonce` that is chosen to satisfy the difficulty rule.

Note: we stated in the instructions above that blocks in a blockchain must satisfy _four_ rules. But these are the only two rules that can be checked by looking at a single block; the other two rules require you to look at the entire blockchain at once, and we will defer those checks to the next question.

You may use the function `isValidTransaction` that you created in question 1. (But before you do so, we strongly recommend checking Question 1 on Gradescope to make sure that your function passes the hidden tests!)

In [52]:
def verifyBlock(block: Block) -> bool:

    # difficulty rule
    block_hash = block.hash()
    difficulty = 1
    if not block_hash.startswith("0" * difficulty):
        return False
    
    for t in block.transactions:
        if isValidTransaction(t) == False or block.nonce == 0:
            return False
    return True
    

In [53]:
grader.check("q2")

q2 results: All test cases passed!

**Question 3: Blockchain validity and balance calculation (6 points).** Write the method `findBalance` that determines if the entire blockchain is valid. 
- If so, then return the balance of the each player in the type of `list[int]`.
- If not, then return `False`.

This method should take two parameters as inputs:
- the `blockchain` that you are checking the validity of, and
- a list of integers containing the initial `players_balance` before the start of the blockchain. Since our toy blockchain doesn't have any mining rewards, instead we initialize the toy blockchain by asserting that Players A and C have one coin each before the start of the system.

Remember that you must check all four properties that determine whether a blockchain is valid:

1. The signature in every transaction is correct,
2. Every block has a `nonce` that is chosen to satisfy the difficulty rule,
3. Every block points to the previous block (and by convention, the initial block -- called the "genesis" block -- has a hash of all-0s), and
4. Everyone has enough money in their account at the moment they post a transaction.

You may use the functions `isValidTransaction` and `verifyBlock` that you created in questions 1 and 2.

In [56]:
players_balance = [1,0,1,0,0] # initially, Players A and C have 1 coin each

def findBalance(blockchain: list[Block], players_balance: list[int]) -> Union[list[int], bool]:
    # input: a blockchain and an integer balance
    # output: either the new list of int, or `False`
    # output type: list[int] or bool

    # rule 3-prev hash
    for i, b in enumerate(blockchain):
        if not verifyBlock(b):
            return False
        if i == 0:
            if b.previous_hash != "0" * 64:
                return False
        else:
            if b.previous_hash != blockchain[i-1].hash():
                return False
        
        for t in b.transactions:
            sender = public_keys.index(t.transaction_payload.sender_public_key)
            receiver = public_keys.index(t.transaction_payload.receiver_public_key)
            amount = t.transaction_payload.amount

            if players_balance[sender] < amount:
                return False
            players_balance[sender] -= amount
            players_balance[receiver] += amount

    return players_balance  
    

In [57]:
grader.check("q3")

q3 results: All test cases passed!

In the remaining questions, your objective is to take on the role of a *miner* and to create new blocks in the toy blockchain. First, you will act as an honest miner and produce a new block legitimately, i.e., extending the end of the `blockchain` that you already have. Second, you will act as a malicious miner and produce a new block illegitimately, by overwriting the prior state on the blockchain. The difficulty level of the toy blockchain is purposely tuned down so that this is possible to do.

**Question 4: Honest mining (3 points).** Mine a third block as player A. Your new block should include a single transaction: as player A, transfer your one and only coin to player E.

You may use the functions `isValidTransaction`, `verifyBlock`, and `findBalance` from above as helper functions within this code, if you wish.

We provide the method `computeSignature` to generate the signature, as player A, corresponding to a transaction payload. This function takes an ECDSA digital signature of the (hash of the) transaction payload. Note that the code uses player A's `secret_key` to produce a signature. (We studied ECDSA in Lecture 4 and in Homework 2; you can review the PyCrypto documentation of ECDSA here if interested: https://pycryptodome.readthedocs.io/en/latest/src/signature/dsa.html?highlight=ecdsa)

In [ ]:
## Execute, but DO NOT MODIFY this code block. ##

def computeSignature(transaction_payload: TransactionPayload):
    # return type: hexadecimal string corresponding to a digital signature
    h = transaction_payload.hash()
    signer = DSS.new(secret_key, 'fips-186-3')
    signature = signer.sign(h)
    return hexlify(signature).decode("ascii")

You have two tasks in this question:

1. Write the method `mineBlock` to create a new block when given any set of transactions. This function should return a valid `Block` object.

2. Write the method `mineHonestBlockThree` that generates a new valid block with required transaction using the `computeSignature` and `mineBlock` functions you wrote above, and appends it to the existing two-block `blockchain`. The function should take the original blockchain as input and return the appended blockchain.

In [ ]:
def mineBlock(blockchain: list[Block], transactions) -> list[Block]:
    # return type: a Block object
    ...

In [ ]:
def mineHonestBlockThree(blockchain: list[Block]) -> list[Block]:
    ...

In [ ]:
grader.check("q4")

**Question 5: Dishonest mining (3 points).** Your objective in this task is to do something malicious: execute a double-spend attack! As player A, you should generate a blockchain that includes a transaction in which you pay **two coins** to player E.

But player A doesn't have 2 coins! So in order to make this attack work, you need to rewrite the state of the blockchain so that player A never pays 1 coin to player B (but still has 1 coin from the start and receives 1 coin from player D, thereby resulting in two coins that can be paid to player E).

Below, you must write code that will produce a new blockchain. Just like before, a blockchain is a list of valid blocks. To satisfy the requirements of this question, your new blockchain must be valid, and it must convince the nodes to switch from the current `blockchain` object to your new, malicious blockchain. Remember that Bitcoin nodes will always prefer the blockchain that has the most work.

You should create a method call `generate_dishonest_blockchain()` that returns a `dishonest_blockchain` (i.e., a list of blocks), which passes all the validity tests even though player A has paid 2 coins to E.

In [ ]:
def generate_dishonest_blockchain() -> list[Block]:
    # use the variable 'dishonest_blockchain' to store your blockchain
    dishonest_blockchain = []
    ...
    return dishonest_blockchain

In [ ]:
grader.check("q5")

### Collaboration Policy

Congratulations on completing the homework! Before you submit the assignment: list all collaborators, sources, and AI tools in accordance with the collaboration policy. Doing so is to your advantage: this is your opportunity to document and explain any external information used and why you believe it adheres to the code of conduct and collaboration policy. If I discover an undocumented violation of the collaboration policy, then this will be considered academic misconduct.

A. Write the names of all classmates you worked with, along with a short description of the work that you performed together.

**Your response:** 

B. List all written materials that you used, such as books or websites (besides the lecture notes and course textbooks). Provide links to any web-based resources, or citations to any physical works.

**Your response:** 

C. State all code that you used from other sources. In particular, if you used an AI tool, then you must include the entire exchange with the AI tool, as per the [CDS Generative AI Assistance Policy](https://www.bu.edu/cds-faculty/culture-community/gaia-policy/).

**Your response:** 

### Sending to Gradescope

After completing the assignment, submit only the `.ipynb` file to Gradescope. It takes a while for the auto grading system to check your work.